In [1]:
# NO code gen + table 


#### import tatqa_utils
import pandas as pd
import table_convert
import pyreadstat
#from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

import utils
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
import numpy as np
from progress.bar import Bar
%load_ext autoreload
%autoreload 2



In [2]:
from langchain_community.chat_models import ChatOllama
from langchain.schema import HumanMessage
llm = ChatOllama(model="qwen3:1.7b")  # vagy bármely más elérhető Ollama modell

In [3]:

from langchain.globals import set_llm_cache
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAI
from langchain_community.cache import SQLiteCache

#llm = ChatOpenAI(temperature=0)
#llm = OpenAI(temperature=0) 
set_llm_cache(SQLiteCache(database_path=".langchain.ollama17.db"))

In [4]:
devdf = pd.read_json('dataset_raw/tatqa_dataset_dev.json')

In [5]:
def get_question(qid):
    for i, item in devdf.iterrows():
        for q in item['questions']:        
            if q['uid'] == qid:
                #table = item['table']['table']
                return (item['table'], q)
    return (None, None)

In [6]:
def gen_code3(llm, messages, question, value_list): 
    
    prompt = ChatPromptTemplate.from_messages(messages)

    output_parser = StrOutputParser()

    chain = prompt | llm | output_parser
    
    response = chain.invoke({"value_list": value_list, "question":question})
    code =  response.replace('```python','').replace('```','')
    
    return (prompt.format(value_list = value_list, question = question), code)

def exec_code(code, value_list):  
        try: 
            loc = locals()   
            if not "run()" in code:
                exec(code + f"\nr = run({value_list})\n", globals(), loc)
            else: 
                exec(code + "\nr = run()\n", globals(), loc)
            return loc['r']
        except Exception as e:
                s = '[Error]'+ str(e)
                print(s)
                return (s,'')

def gen_answer_table(llm, messages, question, table): 
    
    prompt = ChatPromptTemplate.from_messages(messages)

    output_parser = StrOutputParser()

    chain = prompt | llm | output_parser
    
    response = chain.invoke({"table": table, "question":question})
    answer =  response.replace('```python','').replace('```','')
    
    return (prompt.format(table = table, question = question), answer)

In [7]:
def get_answer(llm, messages, table, q):
    try:
        (prompt, res) = gen_answer_table(llm, messages, q, table)
        #print(res)
        res=res.strip()
        #(pred,scale) = res[1:len(res)].split(',')
        (pred,scale) = res[1:len(res)-1].replace("'","").split(',')
        
        pred=pred.strip()
        scale = scale.strip()
        try:
            pred = float(pred)
        except Exception as e:
            None    
        return  (pred, scale, "no code")
    except Exception as e:
        s = '[get_answer Exception]'+ str(e)
        print(s) 
        return  ("", "", "no code")

In [9]:
import prompt_versions
items = [i for idx, i in devdf.iterrows()]
item = items[1]
table = item['table']['table']
q = item.questions[4]
question = q['question']
print(question)
r = get_answer(llm, prompt_versions.prompt_versions['V0'], table, question)
print(r)
#print(r[0], r[1])
#r[1][1:len(res)-1]

What was the change in the amount for Appliances in 2019 from 2018?
[get_answer Exception]too many values to unpack (expected 2)
('', '', 'no code')


In [13]:
##### from tqdm.notebook import tqdm as log_progress
from tqdm.notebook import tqdm as log_progress


devdf = pd.read_json('dataset_raw/tatqa_dataset_dev.json')
cnt = 0
res = []
for i, item in log_progress(devdf.iterrows()):
    try:
        #if cnt == 36:
        #    break;
        #print(item['table']['uid'])
        table = item['table']['table']
        #value_list = table_convert.convert_multitable(table)    
        
        for q in item['questions']:        
            if q['answer_type'] == 'arithmetic' and 'table' == q['answer_from']:   
                cnt = cnt + 1
                #print('BB')
                print(q['uid'])
                _table, _q =  get_question(q['uid'])
                
                table = _table['table']
                
                #[print(r) for r in table]
                
                q = _q['question']
                print(q, end='')
                
                
                r = []
                #for i in [0,1,2]:        
                #   r.append(get_answer(llm, messages, table, q))
                
                #if r[0][0] == r[1][0]:
                #    (v, s) = r[0]
                #elif r[1][0] == r[2][0]:
                #    (v, s) = r[1]
                #elif r[0][0] == r[2][0]:
                #    (v, s) = r[0]
                #else:
                #    (v, s) = (None, None)
                    
                #(v, s, code) = get_answer(llm, messages, table, q)    
                (pred_value, pred_scale, code) = get_answer(llm, prompt_versions.prompt_versions['V0'], table, q)    
                
                if  utils.eval_predicted_value(pred_value, _q['answer']):
                    print("\033[92m Success: " + str(pred_value)+'\033[0m')
                else:    
                    print("\033[91m failure: " + str(pred_value), 'good answer: ', _q['answer'],'\033[0m' )
                
                
                
                if isinstance(pred_value, tuple) and len(pred_value) == 2:
                    print('$$$$')
                    (pred_value, pred_scale) = pred_value
                if pred_scale == "%" or pred_scale == "percentage"  :
                    pred_scale = 'percent'
                if pred_scale not in ["", 'thousand', 'million', 'billion', 'percent']:
                    print('Invalid ', '|',pred_scale,'|')
                    pred_scale = ""    
                #if item['table']['uid'] in res_scale and pred_scale != 'percent':
                #    pred_scale = res_scale[item['table']['uid']]
                err=""
                if isinstance(pred_value, str):
                    #print("string")
                    if  pred_value.startswith('[Error]'):
                        (pred_value, pred_scale) = ("", "")
                        err = v                                                        
                res.append(({"answer_type":_q["answer_type"], "answer": _q["answer"], 'scale': _q["scale"]}, pred_value, pred_scale, _q, code, table,"", err))
    except Exception as e:
        s = '[Outer Exception]'+ str(e)
        print(s)

0it [00:00, ?it/s]

b2786c1a-37de-4120-b03c-32bf5c81f157
What was the change in the amount for Appliances in 2019 from 2018?[get_answer Exception]too many values to unpack (expected 2)
 failure:  good answer:  -94 
fe11f001-3bfe-4089-8108-412676f0a780
What was the percentage change in the amount for Appliances in 2019 from 2018?[get_answer Exception]too many values to unpack (expected 2)
 failure:  good answer:  -12.14 
5103aed0-b4e8-4fae-bf78-e2c9f4ba84cf
What is the difference between the domestic and international discount rates as at September 30, 2019?[get_answer Exception]too many values to unpack (expected 2)
 failure:  good answer:  2.1 
4dc8be43-d8d9-4b08-9ffd-9c19012361ce
What is the year on year percentage change in domestic discount rate between 2018 and 2019?[get_answer Exception]too many values to unpack (expected 2)
 failure:  good answer:  6.67 
6c44a1a8-0785-43a0-90ab-7e21df2c57d9
What is the year on year percentage change in international expected return on plan assets between 2018 and 2

KeyboardInterrupt: 

In [ ]:
from tatqa_metric import TaTQAEmAndF1

metrics = TaTQAEmAndF1()

for ans, pred, pred_scale, _,_, _,_,_ in res:
    metrics(ans, pred, pred_scale)
pred_em, pred_f1, scale_score, op_score = metrics.get_overall_metric(reset=False)
print( pred_em, pred_f1, scale_score)

In [ ]:
import re
res2 = []
for ans, pred, pred_scale, q,code, table,value_list,err in res:
    good = utils.eval_predicted_value(pred, ans['answer'])
    regex = re.compile("\\(\\d+,?\\d+", re.VERBOSE)    
    match = regex.search(str(table))
    
    match = match != None
    res2.append( {'qid': q['uid'], 'question' : q['question'], 'derivation': q['derivation'], 'pred' : pred, 'pred_scale': pred_scale,  'answer': ans['answer'],  'scale':  ans['scale'],  'abs': 'abs(' in code, 'inv' : None,  'good': good, 'parenth': match})
res2 = pd.DataFrame(res2)
print(len(res2.query('good == True'))/len(res2))


In [31]:
res2.to_csv('exp43_V0.2_res.csv')

In [36]:
print('-1')
print("full: ", len(res2), '\ngood: ', len(res2.query('good == True')),  '\nABS:', len(res2.query('abs == True')), '\nABS Good:', len(res2.query('abs == True and good == True')),
      '\nABS Bad:', len(res2.query('abs == True and good == False')), '\nABS Bad Inv:', len(res2.query('abs == True and inv == True')), 
      '\nNON ABS Good:', len(res2.query('abs == False and good == True')),'\nNON ABS Bad:', len(res2.query('abs == False and good == False')), '\nNON ABS Bad Inv:', len(res2.query('abs == True and inv == True')), 
      '\nPAR good: ', len(res2.query('good == True and parenth ==True')),    '\nPAR bad: ', len(res2.query('good == False and parenth ==True')), 
      '\nPAR ABS:', len(res2.query('abs == True  and parenth ==True'  )), '\nPAR ABS Good:', len(res2.query('abs == True and good == True and parenth ==True')),
      '\nPAR ABS Bad:', len(res2.query('abs == True and good == False  and parenth ==True')), '\nPAR ABS Bad Inv:', len(res2.query('abs == True and inv == True and parenth ==True')),
      '\nPAR NON ABS:', len(res2.query('abs == False  and parenth ==True'  )), '\nPAR NON ABS Good:', len(res2.query('abs == False and good == True and parenth ==True')),
      '\nPAR NON ABS Bad:', len(res2.query('abs == False and good == False  and parenth ==True')), '\nPAR Non ABS Bad Inv:', len(res2.query('abs == False and inv == True and parenth ==True')),

      
     )

-1
full:  485 
good:  165 
ABS: 0 
ABS Good: 0 
ABS Bad: 0 
ABS Bad Inv: 0 
NON ABS Good: 165 
NON ABS Bad: 320 
NON ABS Bad Inv: 0 
PAR good:  58 
PAR bad:  182 
PAR ABS: 0 
PAR ABS Good: 0 
PAR ABS Bad: 0 
PAR ABS Bad Inv: 0 
PAR NON ABS: 240 
PAR NON ABS Good: 58 
PAR NON ABS Bad: 182 
PAR Non ABS Bad Inv: 10


In [37]:
res2.query('abs == True and inv == True  and parenth ==True')

,qid,question,derivation,pred,pred_scale,answer,scale,abs,inv,good,parenth


In [38]:
res2.query("question.str.contains('the change')")

,qid,question,derivation,pred,pred_scale,answer,scale,abs,inv,good,parenth
0,b2786c1a-37de-4120-b03c-32bf5c81f157,What was the change in the amount for Applianc...,680-774,-94.00,million,-94.00,million,False,False,True,False
12,78fc6d55-c20c-4f71-99fe-bc40a16e61d0,What is the change of IMFT’s total assets from...,"2,664 - 3,025",-361.00,million,-361.00,,False,False,True,False
16,22e20f25-669a-46b9-8779-2768ba391955,What is the change between 2018 and 2019 avera...,"[(4,411+4,044)/2] -[(4,044+3,316)/2]",367.00,million,547.50,million,False,False,False,True
24,65ec782c-691e-45df-b541-caecb85154ff,What was the change in Capital redemption rese...,0.7-0.5,0.20,million,0.20,million,False,False,True,True
32,79f06004-f4fc-4e82-a9fe-3c389a2f81b6,What was the change in the statutory federal i...,21.0% - 21.0%,0.00,percent,0.00,percent,False,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...
455,ba16499f-3c96-4faa-81b0-874f9ace7c8c,What was the change in the basic net income pe...,0.43-1.75,3.82,,-1.32,,False,False,False,False
456,9d2aa4d9-194f-417a-9ed7-95cf4f5c95cd,What is the change in accumulated depreciation...,"(2,964.6 - 2,755.1)/2,755.1",-209.50,million,7.60,percent,False,False,False,True
458,14120019-ba0a-483f-9046-2f2d4118761c,How much is the change in additions of financi...,"39,116-51,185",12.93,,-12069.00,million,False,False,False,True
460,bb6966e0-61e6-44c6-87ad-69f952453829,How much is the change between 2018 and 2019 c...,"1,740-4,946",3.21,billion,-3206.00,million,False,False,False,True


# Single case

In [15]:
qid = '732c81f8-a16d-4d34-9917-fa98c195feec'
qid = 'e4f21617-1a84-40e4-bdf0-671df88cb862'
qid = '64c902c6-f426-4432-84b3-c10b3065716f'
qid = '524463b6-b8d7-410d-926a-699f7b7cdd99'
qid = '147f7c09-a627-4719-93ad-33661b5a4543'
#qid = '710bc8e8-3a6b-42ef-b122-a348250715f1'
#qid='a81f1322-e74f-4e3c-a6cf-4b8d25d01cf5'
#qid = '4190424e-ce6e-4f5c-9175-51afdba156e5'
messages = prompt_versions.prompt_versions['V0']

_table, _q =  get_question(qid)
table = _table['table']
print("tid: ",_table['uid'])
[print(r, ',') for r in table]

q = _q['question']
print(q)
#values = table_convert.convert_multitable(table)

#[print(v) for v in values]
#p, code = gen_code_table(llm, prompt_versions.prompt_versions['V0'], q, table)
#print(code)

#(pred, s) = exec_code_table(code, table)
(prompt, res) = gen_answer_table(llm, prompt_versions.prompt_versions['V0'], q, table)
print(res)
(pred,pred_scale) = res[1:len(res)-1].replace("'","").split(',')
print(pred,pred_scale)
pred = float(pred)

llimit = _q['answer']*0.9999
ulimit = _q['answer']*1.0001
#if isinstance(v, float) and ( v == _q['answer'] or (v > _q['answer']*rng and v < _q['answer']rng)):
good = isinstance(pred, float) and ((pred > 0 and llimit < pred and pred < ulimit) or (pred < 0 and llimit > pred and pred > ulimit) or pred==ulimit or pred == llimit)      
if good:
    print("\033[92m Success: " + str(pred)+'\033[0m')
else:    
    print("\033[91m failure: " + str(pred), 'good answer: ', _q['answer'],'\033[0m' )  

tid:  db62468f-cfd5-4b89-b2b0-eb9d1a78a31e
['', '', 'Year Ended May 31,', ''] ,
['(Dollars in millions)', '2019', 'Change', '2018'] ,
['Net cash provided by operating activities', '$14,551', '-5%', '$15,386'] ,
['Net cash provided by (used for) investing activities', '$26,557', '572%', '$(5,625)'] ,
['Net cash used for financing activities', '$(42,056)', '321%', '$(9,982)'] ,
What is the difference between the net cash used for financing activities in 2018 and 2019?
(32.07, million)
32.07  million
 failure: 32.07 good answer:  -32074 


In [180]:
pd.DataFrame(table)

,0,1,2,3,4
0,,F19,F18 (3),,CHANGE
1,$ MILLION,53 WEEKS,52 WEEKS,CHANGE,NORMALISED
2,Sales,"8,657","8,244",5.0%,3.2%
3,EBITDA,579,603,(4.1)%,(5.4)%
4,Depreciation and amortisation,(105),(87),20.1%,20.1%
5,EBIT,474,516,(8.2)%,(9.7)%
6,Gross margin (%),22.9,23.1,(16) bps,(14) bps
7,Cost of doing business (%),17.4,16.8,63 bps,64 bps
8,EBIT to sales (%),5.5,6.3,(78) bps,(78) bps
9,Sales per square metre ($)$),"18,675","18,094",3.2%,1.4%


In [303]:
import prompt_versions

v1res = pd.read_csv('exp42_V1_res.csv')
qids = list(v1res.query('good == False')['qid'])

messages = prompt_versions.prompt_versions['V9']
#qids= list(set(qids))
goods = 0
for qid in qids:
    _table, _q =  get_question(qid)
    table = _table['table']
    #print("tid: ",_table['uid'])
    #[print(r) for r in table]

    q = _q['question']
    #print(q)
  
    values = table_convert.convert_multitable(table)
    #[print(v) for v in values]
    p, code = gen_code3(llm, messages, q, values)
    #print(len(p) + len(code))
    (pred, s) = exec_code(code, values)
    llimit = _q['answer']*0.9999
    ulimit = _q['answer']*1.0001
    #if isinstance(v, float) and ( v == _q['answer'] or (v > _q['answer']*rng and v < _q['answer']rng)):
    good = isinstance(pred, float) and ((pred > 0 and llimit < pred and pred < ulimit) or (pred < 0 and llimit > pred and pred > ulimit) or pred==ulimit or pred == llimit)      
    if good:
        goods = goods + 1
        print(_q['uid'],"\033[92m Success: " + str(pred)+'\033[0m')
    else:    
        print(_q['uid'],"\033[91m failure: " + str(pred), 'good answer: ', _q['answer'],'\033[0m' ) 
print(goods/len(qids))

a983501d-2eec-486d-9661-e520c7c8af5e  Success: 3728.0
64c902c6-f426-4432-84b3-c10b3065716f  failure: 16458.0 good answer:  -16458 
(14212.0, '')
010df393-a62e-408d-888a-045a4e435e6a  Success: 14212.0
[Error]
5c8c999e-354f-4693-9b2d-29e3c03cb2af  failure: [Error] good answer:  -9.9 
94ef7822-a201-493e-b557-a640f4ea4d83  Success: 56.0
bfce0375-cbd0-4e25-9b82-0424058918f1  failure: 12136.91 good answer:  12122 
73693527-ed4b-4d07-941e-0e654095a43d  failure: 0.06 good answer:  0.9500000000000001 
59f13f19-4494-4c54-8e9a-1b499d50021a  failure: 2070862713946.92 good answer:  981341.34 
[Error]
b382a11b-749b-425a-a77d-20e943e00f77  failure: [Error] good answer:  0.11 
147f7c09-a627-4719-93ad-33661b5a4543  failure: 2.0 good answer:  -2 
f1034ee7-b182-48d5-bad8-1d9902500f57  Success: 1226114.0
f36e5912-f63c-4837-a9ed-ddbe73e5148b  failure: 6.01 good answer:  12.02 
eb50d3aa-16da-4087-9ab2-10106ebd10e7  failure: 4859.25 good answer:  2988 
f1d5753b-05a9-4752-b03a-493e97804fb7  failure: 310611.5 

In [299]:
qids= [
    #bad
    #perc change
    '732c81f8-a16d-4d34-9917-fa98c195feec',
    'e4f21617-1a84-40e4-bdf0-671df88cb862',
    '359f9c4c-b405-40e5-88b5-86bfa0069b89',
    '6f65d33d-c721-45f9-9519-9ea4f5d7e740',

    #diff
    '64c902c6-f426-4432-84b3-c10b3065716f',
    '524463b6-b8d7-410d-926a-699f7b7cdd99',

    #change
    '147f7c09-a627-4719-93ad-33661b5a4543',
    
    #good
    #perc change
    'fe11f001-3bfe-4089-8108-412676f0a780',
    '4dc8be43-d8d9-4b08-9ffd-9c19012361ce',
    'bed1fce2-69cb-4d1e-a34a-01950a1770bd',

    #diff
    '5103aed0-b4e8-4fae-bf78-e2c9f4ba84cf',
    '8fb4b76c-b4d9-40e9-8233-28a19d06df41',

    #change
    'c9b0fa1e-6fe7-403e-b0f1-9367e6485fcf',
    'bb6966e0-61e6-44c6-87ad-69f952453829'
    
]
for qid in qids:
    _table, _q =  get_question(qid)
    table = _table['table']
    #print("tid: ",_table['uid'])
    #[print(r) for r in table]

    q = _q['question']
    #print(q)
    print(_q['uid'])
    values = table_convert.convert_multitable(table)
    #[print(v) for v in values]
    p, code = gen_code3(llm, messages, q, values)
    #print(len(p) + len(code))
    (v, s) = exec_code(code, values)
    rng = 0.01
    if isinstance(v, float) and ( v == _q['answer'] or (v > _q['answer']-rng and v < _q['answer']+rng)):
        print("\033[92m Success: " + str(v)+'\033[0m')
    else:    
        print("\033[91m failure: " + str(v), 'good answer: ', _q['answer'],'\033[0m' ) 

732c81f8-a16d-4d34-9917-fa98c195feec
 Success: 103.67
e4f21617-1a84-40e4-bdf0-671df88cb862
 Success: -52.63
359f9c4c-b405-40e5-88b5-86bfa0069b89
 Success: 185.3
6f65d33d-c721-45f9-9519-9ea4f5d7e740
 Success: 368.13
64c902c6-f426-4432-84b3-c10b3065716f
 failure: 16458.0 good answer:  -16458 
524463b6-b8d7-410d-926a-699f7b7cdd99
 failure: 32074.0 good answer:  -32074 
147f7c09-a627-4719-93ad-33661b5a4543
 failure: 2.0 good answer:  -2 
fe11f001-3bfe-4089-8108-412676f0a780
 Success: -12.14
4dc8be43-d8d9-4b08-9ffd-9c19012361ce
 Success: 6.67
bed1fce2-69cb-4d1e-a34a-01950a1770bd
 Success: 26.82
5103aed0-b4e8-4fae-bf78-e2c9f4ba84cf
 Success: 2.1
8fb4b76c-b4d9-40e9-8233-28a19d06df41
(835.0, '')
 failure: 835.0 good answer:  -835 
c9b0fa1e-6fe7-403e-b0f1-9367e6485fcf
 failure: 48207.0 good answer:  -48207 
bb6966e0-61e6-44c6-87ad-69f952453829
 failure: 3206.0 good answer:  -3206 
